## PIP install necessary libraries

In [ ]:
!pip install tensorflow


In [ ]:
!pip install keras

## Load the dataset using load from tensorflow and preprocess the data

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Load KMNIST dataset
(ds_train, ds_test), ds_info = tfds.load(
    'kmnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Define preprocessing function
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixels to [0, 1]
    image = tf.reshape(image, [-1])  # Flatten image to 1D array
    label = tf.one_hot(label, depth=10)  # One-hot encode labels
    return image, label

# Apply preprocessing to the datasets
ds_train = ds_train.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Access a single batch to verify the preprocessing
for images, labels in ds_train.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/kmnist/incomplete.TMWSGJ_3.0.1/kmnist-train.tfrecord*...:   0%|          |…

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/kmnist/incomplete.TMWSGJ_3.0.1/kmnist-test.tfrecord*...:   0%|          | …

Dataset kmnist downloaded and prepared to /root/tensorflow_datasets/kmnist/3.0.1. Subsequent calls will reuse this data.
Image batch shape: (32, 784)
Label batch shape: (32, 10)


#Model Summary of three optimizers

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, AdamW

def create_model(optimizer):
    model = Sequential([
        Dense(256, activation='relu', input_shape=(28*28,)),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Create instances of the model with different optimizers
model_adam = create_model(optimizer=Adam(learning_rate=0.001))
model_rmsprop = create_model(optimizer=RMSprop(learning_rate=0.001))
model_adamw = create_model(optimizer=AdamW(learning_rate=0.001))

# Print summaries of the model architectures
print("Adam Optimizer Model Summary:")
model_adam.summary()

print("\nRMSprop Optimizer Model Summary:")
model_rmsprop.summary()

print("\nAdamW Optimizer Model Summary:")
model_adamw.summary()

Adam Optimizer Model Summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               200960    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
Total params: 235146 (918.54 KB)
Trainable params: 235146 (918.54 KB)
Non-trainable params: 0 (0.00 Byte)
____________________________________________________

### Defining the model Architecture, creating the model and performing hyperparameter tuning -  dropout layers and learning rates, and then train the model


#### In the below code, we experimented without using any search techniques to check the validation accuracy and loss

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, AdamW
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_datasets as tfds

# Load KMNIST dataset
(ds_train, ds_test), ds_info = tfds.load(
    'kmnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Define preprocessing function
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixels to [0, 1]
    image = tf.reshape(image, [-1])  # Flatten image to 1D array
    label = tf.one_hot(label, depth=10)  # One-hot encode labels
    return image, label

# Apply preprocessing to the datasets
ds_train = ds_train.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Define model architecture
def create_model():
    model = Sequential([
        Dense(256, activation='relu', input_shape=(28*28,)),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(10, activation='softmax')
    ])
    return model

# Optimizers to compare
optimizers = {
    'Adam': Adam(learning_rate=0.001),
    'RMSprop': RMSprop(learning_rate=0.001),
    'AdamW' : AdamW(learning_rate=0.001)
}

results = {}

# Iterate over optimizers
for name, optimizer in optimizers.items():
    print(f'Training with {name}')

    # Create and compile the model
    model = create_model()
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Early stopping callback
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

    # Train the model with validation data
    history = model.fit(
        ds_train,
        epochs=50,
        validation_data=ds_test,  # Use ds_test as validation data
        callbacks=[early_stopping]
    )

    # Evaluate on test set
    test_loss, test_accuracy = model.evaluate(ds_test)
    # Extract validation accuracy from history
    validation_accuracy = history.history['val_accuracy'][-1]

    # Store results
    results[name] = {
        'history': history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'validation_accuracy': validation_accuracy
    }

    print(f'Test accuracy with {name}: {test_accuracy:.4f}')
    print(f'Validation accuracy with {name}: {validation_accuracy:.4f}')
# Print final results
print("\nResults:")
for name, result in results.items():
    print(f"{name}: Test Accuracy = {result['test_accuracy']:.4f}, Validation Accuracy = {result['validation_accuracy']:.4f}")

Training with Adam
Epoch 1/50
1875/1875 [==============================] - 13s 6ms/step - loss: 0.4204 - accuracy: 0.8702 - val_loss: 0.4838 - val_accuracy: 0.8495
Epoch 2/50
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2136 - accuracy: 0.9358 - val_loss: 0.3964 - val_accuracy: 0.8840
Epoch 3/50
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1608 - accuracy: 0.9515 - val_loss: 0.3972 - val_accuracy: 0.8893
Epoch 4/50
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1378 - accuracy: 0.9568 - val_loss: 0.3599 - val_accuracy: 0.8995
Epoch 5/50
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1131 - accuracy: 0.9640 - val_loss: 0.3681 - val_accuracy: 0.9024
Epoch 6/50
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1004 - accuracy: 0.9682 - val_loss: 0.3605 - val_accuracy: 0.9096
Epoch 7/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0875 - accuracy: 0.9717 - val_loss: 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Defining the model Architecture, creating the model and performing hyperparameter tuning - RandomSearchCV, dropout layers and learning rates, and then train the model

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop, AdamW
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import HTML, display
import imageio

# Function to preprocess KMNIST dataset
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixels to [0, 1]
    image = tf.reshape(image, (28, 28, 1))     # Reshape to 28x28x1 (for convolutional layers)
    label = tf.one_hot(label, depth=10)        # One-hot encode labels
    return image, label

# Load KMNIST dataset
(ds_train, ds_test), ds_info = tfds.load(
    'kmnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Apply preprocessing to the datasets
ds_train = ds_train.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Define model architecture with enhanced features
def create_model(optimizer=Adam(learning_rate=0.001), dropout_rate=0.2):
    model = Sequential([
        Flatten(input_shape=(28, 28, 1)),      # Flatten layer for convolutional networks
        Dense(256, activation='relu'),
        Dropout(dropout_rate),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Optimizers to compare
optimizers = {
    'Adam': Adam(learning_rate=0.001),
    'RMSprop': RMSprop(learning_rate=0.001),
    'AdamW': AdamW(learning_rate=0.001)
}

# Grid search or random search setup
# Example grid for demonstration purposes; adjust according to your needs
param_grid = {
    'optimizer': [Adam(), RMSprop(), AdamW()],
    'dropout_rate': [0.2, 0.3, 0.4]
}
#search = GridSearchCV(estimator=create_model(), param_grid=param_grid, cv=3, verbose=1)

search = RandomizedSearchCV(estimator=create_model(), param_distributions=param_grid, n_iter=3, cv=3, verbose=1)

# Train models with different optimizers and hyperparameters
results = {}
for name, optimizer in optimizers.items():
    print(f'Training with {name}')
    model = create_model(optimizer=optimizer)

    # Early stopping callback
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

    # Train the model with validation data
    history = model.fit(
        ds_train,
        epochs=50,
        validation_data=ds_test,
        callbacks=[early_stopping]
    )

    # Evaluate on test set
    test_loss, test_accuracy = model.evaluate(ds_test)

    # Store results
    results[name] = {
        'model': model,
        'history': history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy
    }

    print(f'Test accuracy with {name}: {test_accuracy:.4f}')

# Print final results
print("\nResults:")
for name, result in results.items():
    print(f"{name}: Test Accuracy = {result['test_accuracy']:.4f}")


Training with Adam
Epoch 1/50
1875/1875 [==============================] - 15s 7ms/step - loss: 0.4214 - accuracy: 0.8687 - val_loss: 0.4689 - val_accuracy: 0.8576
Epoch 2/50
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2164 - accuracy: 0.9333 - val_loss: 0.4155 - val_accuracy: 0.8777
Epoch 3/50
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1614 - accuracy: 0.9505 - val_loss: 0.3960 - val_accuracy: 0.8837
Epoch 4/50
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1362 - accuracy: 0.9575 - val_loss: 0.3760 - val_accuracy: 0.8959
Epoch 5/50
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1149 - accuracy: 0.9644 - val_loss: 0.3458 - val_accuracy: 0.9038
Epoch 6/50
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0985 - accuracy: 0.9688 - val_loss: 0.3888 - val_accuracy: 0.9003
Epoch 7/50
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0921 - accuracy: 0.9704 - val_loss

## Plot the loss functions according to the optimizers

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import HTML, display

def plot_results(results):
    # Plot accuracy
    fig_accuracy = go.Figure()

    for name, result in results.items():
        history = result['history']
        fig_accuracy.add_trace(go.Scatter(x=list(range(1, len(history.history['accuracy']) + 1)),
                                          y=history.history['accuracy'],
                                          mode='lines',
                                          name=f'{name} Train'))

        fig_accuracy.add_trace(go.Scatter(x=list(range(1, len(history.history['val_accuracy']) + 1)),
                                          y=history.history['val_accuracy'],
                                          mode='lines',
                                          name=f'{name} Validation'))

    fig_accuracy.update_layout(title='Model Accuracy',
                               xaxis_title='Epochs',
                               yaxis_title='Accuracy',
                               legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
                               margin=dict(l=0, r=0, t=30, b=0))

    # Plot loss
    fig_loss = go.Figure()

    for name, result in results.items():
        history = result['history']
        fig_loss.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)),
                                      y=history.history['loss'],
                                      mode='lines',
                                      name=f'{name} Train'))

        fig_loss.add_trace(go.Scatter(x=list(range(1, len(history.history['val_loss']) + 1)),
                                      y=history.history['val_loss'],
                                      mode='lines',
                                      name=f'{name} Validation'))

    fig_loss.update_layout(title='Model Loss',
                           xaxis_title='Epochs',
                           yaxis_title='Loss',
                           legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
                           margin=dict(l=0, r=0, t=30, b=0))

    # Display interactive plots
    display(HTML('<h2>Model Accuracy</h2>'))
    display(fig_accuracy)

    display(HTML('<h2>Model Loss</h2>'))
    display(fig_loss)

# Example usage
plot_results(results)
